In [94]:
# 시스템 모듈
import os
from dotenv import load_dotenv

# 랭체인 모듈
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory 
from langchain_core.prompts import MessagesPlaceholder

# 환경변수 로드
load_dotenv()

# 벡터 DB 설정
DB_INDEX = "MANUAL_DB"
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local(DB_INDEX, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# LLM 모델 설정
llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0,
    streaming=True,
)

# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system",
    """
    Answer the question using ONLY the following context.
    If you don't know the answer just say you don't know. DON'T make anything up.(!very important)
    Provide all responses in Korean.

    If the context includes metadata like a source link, provide this link at the end of the answer using the format [link_text](link).
    If there are prerequisites in the context, mention them in the response, using the format [title][link].

    Answer the user's question in three different ways depending on the complexity of the request and the specificity needed:

    1. **Concise Answer**: If the question is simple and straightforward, and the answer is available in the context, provide a brief answer, typically in 1-3 sentences. If the information is not available in the context, say you don't know. Example:
       - User: "What is the warranty period for this product?"
       - Assistant: "The warranty period is 2 years from the date of purchase. For more details, check the warranty card included with your product. Do you need more help or have other questions?"
       - If not available in the context: "I'm sorry, but I don't know the warranty period as it is not mentioned in the provided context."

    2. **Step-by-Step Guide**: If the question involves a process and the steps are available in the context, provide a detailed step-by-step guide. Start with the first step and then check if the user needs further assistance before proceeding. If the process details are not in the context, state that you don't know. Example:
       - User: "How do I set up the new router?"
       - Assistant: "To set up your new router, follow these steps:
         1. Unbox the router and check that all necessary components are present.
         2. Connect the router to your modem using the provided Ethernet cable.
         3. Plug in the power adapter and turn on the router.
         Would you like to continue with the setup process, or do you need help with any of these steps?"
       - If not available in the context: "I'm sorry, but the setup steps for the router are not provided in the context. Please consult the product manual or contact support for assistance."

    3. **Request for Specificity**: If the question is too broad or if there are multiple possible answers, and the context does not specify enough details, ask the user to provide more specific information. If the context does not cover the details needed to refine the question, acknowledge the limitation. Example:
       - User: "How do I add a user?"
       - Assistant: "There are different ways to add a user depending on the context. For instance, adding a user to a workspace, a gallery, or a project each involves different steps. Could you please specify where you would like to add the user? This will help me provide the most accurate instructions."
       - If not enough context: "I'm sorry, but the context provided does not specify the details needed to guide you in adding a user. Please provide more specific information or consult the relevant documentation."

    ---
    
    Context:{context}
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])


# 검색 증강 문서 처리
def formatting_docs(docs):
    unique_sources = set()
    context = []
    for doc in docs:
        source = doc.metadata.get("source", "")
        if source not in unique_sources and os.path.exists(source):
            unique_sources.add(source)
            with open(source, 'r', encoding='utf-8') as file:
                content = file.read()
                context.append(content)
    
    return "\n\n".join(context)

# 대화 메모리 설정
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# 체인 설정
chain = {
    "context": retriever | RunnableLambda(formatting_docs),
    "chat_history": RunnableLambda(lambda _: memory.load_memory_variables({}).get('chat_history', [])),
    "question": RunnablePassthrough()
} | prompt | llm 

# 대화 메모리를 사용한 입력 및 출력 관리
def run_chain_with_memory(query):
    # 체인을 사용하여 응답 생성
    response = chain.invoke(query)
    # 대화 메모리에 응답 추가
    memory.save_context(
        inputs={"human": query},
        outputs={"ai": response.content}
    )
    return response


query = "새로운 저장소 생성하는법"
response = run_chain_with_memory(query)

print(response)


content='### 간단한 답변\n로컬에서 새로운 Git 저장소를 생성하려면, 먼저 프로젝트 디렉토리를 만들고 `git init` 명령을 실행하세요. GitHub나 GitLab에서 저장소를 생성하려면 해당 플랫폼에 로그인하여 "New repository" 옵션을 선택하고 필요한 정보를 입력한 후 저장소를 생성하세요.\n\n### 단계별 가이드\n#### 로컬에서 새 저장소 생성하기\n1. 프로젝트를 시작할 새 디렉토리를 생성하고 해당 디렉토리로 이동합니다:\n   ```bash\n   mkdir my_project\n   cd my_project\n   ```\n2. 다음 명령을 실행하여 Git 저장소를 초기화합니다:\n   ```bash\n   git init\n   ```\n   이 명령은 `.git`이라는 하위 디렉토리를 생성합니다. 이 디렉토리에는 저장소에 필요한 모든 Git 메타데이터가 포함됩니다.\n\n3. 초기화 후에는 다음과 같은 작업을 수행할 수 있습니다:\n   - 파일 생성 또는 기존 파일 추가\n   - `git add` 명령으로 파일을 스테이징\n   - `git commit` 명령으로 변경사항 커밋\n\n#### GitHub, GitLab 등에서 새 저장소 생성하기\n1. 해당 플랫폼의 웹사이트에 로그인합니다.\n2. "New repository" 또는 유사한 옵션을 선택합니다.\n3. 저장소 이름, 설명, 공개/비공개 설정 등을 입력합니다.\n4. "Create repository" 버튼을 클릭합니다.\n5. 생성된 저장소를 로컬 시스템에 클론합니다:\n   ```bash\n   git clone https://github.com/username/repository.git\n   ```\n\n### 추가 정보\n더 자세한 내용은 [저장소 생성은 어떻게 하나요?](https://git-scm.com/book/en/v2/Git-on-the-Server-Getting-Git-on-a-Server) 링크를 참고하세요.' response_

In [95]:
query = "깃의 역사에 대해서 알려줘요"
response = run_chain_with_memory(query)

print(response)

content='죄송하지만, 제공된 문맥에는 Git의 역사에 대한 정보가 포함되어 있지 않습니다. Git의 역사에 대해 알고 싶다면, Git의 공식 문서나 관련 자료를 참고하시기 바랍니다.' response_metadata={'finish_reason': 'stop'} id='run-91698722-e291-43ba-945c-a43d1668b474-0'


In [77]:
print(memory.load_memory_variables({}).get('chat_history'))



[HumanMessage(content='김장호가 누구인가요?'), AIMessage(content='죄송하지만, 제공된 정보에는 김장호에 대한 언급이 없습니다. 추가 정보가 필요하다면 다른 출처를 참고하시기 바랍니다.'), HumanMessage(content='git이 무엇인가요?'), AIMessage(content='Git은 Linus Torvalds가 2005년에 개발한 분산 버전 관리 시스템(DVCS)입니다. 소프트웨어 개발 프로세스에서 소스 코드의 변경사항을 효과적으로 추적하고 관리하는 데 사용됩니다. 주요 특징으로는 분산 버전 관리, 강력한 브랜칭 및 병합 기능, 데이터 무결성 보장, 스테이징 영역 제공, 그리고 빠른 성능 등이 있습니다. Git은 중앙 집중식 버전 관리 시스템과 달리 오프라인 작업이 가능하고, 더 빠른 브랜칭 및 병합을 지원하며, 전체 프로젝트 히스토리에 대한 로컬 액세스를 제공합니다.'), HumanMessage(content='내가 방금전에 어떤 내용을 물어보았나요?'), AIMessage(content='당신은 "김장호가 누구인가요?"라고 물어보셨습니다.')]
